In [2]:
from tensorflow.keras.models import load_model
import sys
print(sys.executable)
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from time import time
import pandas as pd
from glob import glob
from KerasOCRTrainer2 import *
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, optimizers
from tensorflow import keras
from tensorflow.keras.layers import Lambda
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import cv2
from PIL import Image
from random import shuffle

/Users/eduard/.pyenv/versions/3.6.8/envs/keras36/bin/python


In [4]:
model = load_model('models/ep46-va0.36.hdf5')

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 24, 24, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 24, 24, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 32)   25632       dropout[0][0]                    
______________________________________________________________________________________________

In [6]:
input_img = model.get_layer('input_1').input
num1 = model.get_layer('num1').output
num2 = model.get_layer('num2').output
num3 = model.get_layer('num3').output
out = concatenate([num1,num2,num3],axis=1)
model2 = Model(inputs=input_img,outputs = out)
# model2.compile(optimizers.Adam(lr=0.001),loss=["categorical_crossentropy"])

In [7]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 24, 24, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 24, 24, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 12, 12, 32)   25632       dropout[0][0]                    
______________________________________________________________________________________________

In [9]:
acc = .95
model_name =     f'recombined_models/dragon_mini_32_48_{int(acc*100)}.h5'
tf_lite_model_name = f'tflite_models/dragon_mini_32_48_{int(acc*100)}.tflite'
model2.save(model_name)

In [10]:
# Convert to TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)
tf_lite_model = converter.convert()
open(tf_lite_model_name, "wb").write(tf_lite_model)

W0823 08:36:00.835426 4770354624 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W0823 08:36:00.854777 4770354624 deprecation.py:323] From /Users/eduard/.pyenv/versions/3.6.8/envs/keras36/lib/python3.6/site-packages/tensorflow/lite/python/util.py:238: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0823 08:36:00.855683 4770354624 deprecation.py:323] From /Users/eduard/.pyenv/versions/3.6.8/envs/keras36/lib/python3.6/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


369752

In [11]:
def reverse_one_hot(one_hot):
    one_hot = one_hot.reshape(3,11)
    categories = [' ','0','1','2','3','4','5','6','7','8','9']
    ix0 = one_hot[0].tolist().index(1)
    ix1 = one_hot[1].tolist().index(1)
    ix2 = one_hot[2].tolist().index(1)
    res = categories[ix0]+categories[ix1]+categories[ix2]
    return res

In [12]:
file_list = glob('/Users/eduard/workspaces/ml_projects/keras/VitmoOCR/data2/*/*.png')
shuffle(file_list)

interpreter = tf.compat.v2.lite.Interpreter(model_path=tf_lite_model_name)
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on random input data.
input_shape = input_details[0]['shape']
print(f'input tensor shape: {input_shape}')

files = file_list
shuffle(files)
incorrect_classes = []
all_classes = []
n_correct = 0
n_incorrect = 0

for ix, file in enumerate(files):

    img = Image.open(file)
    img = img.resize((48, 48))
    np_img = np.array(img, dtype=np.float32).reshape(*input_shape)
    img_mean = np.mean(np_img)
    img_std = np.std(np_img)

    np_img = np_img/255
    interpreter.set_tensor(input_details[0]['index'], np_img)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    output_data = (output_data>0.5).astype(int)
    try:
        res = reverse_one_hot(output_data)
    except:
#         print(f'rev_oh failed for {file}')
        continue
    lable = file.split('/')[-2].rjust(3)
#     if res=='   ' or lable=='nan':
#         print(f'res: |{res}|, lab: |{lable}|')
    if res=='   ':
        res = 'nan'


    if res == lable:
        n_correct = n_correct+1
    else:
        n_incorrect = n_incorrect+1
    acc = n_correct/(n_correct+n_incorrect)
    print(f'\r ix:{ix} acc: {acc}',end='')
    


input tensor shape: [ 1 48 48  3]
 ix:147061 acc: 0.9490578663755151

In [10]:
output_details

[{'name': 'concatenate/concat',
  'index': 0,
  'shape': array([ 1, 33], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]